In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# sheets_list = ['kino.mail', 'dom.mail', 'health.mail']
# classes_list = ['Культура и Искусство', 'Строительство и Интерьер', 'Красота и Здоровье']
# datasets_list = []
# for i, target in enumerate(sheets_list):
#     current_dataset = pd.read_excel('data.mail.xlsx', sheet_name=target)
#     current_dataset['target'] = classes_list[i]
#     datasets_list.append(current_dataset)

In [3]:
my_df = pd.read_csv('marked_dataset.csv')
my_df

,text,target
0,"Она меня завораживает. Я бросаю все и всех, ка...",Развлечение
1,«Рынок азиатских продуктов может выиграть в ре...,Развлечение
2,Cafezinho do BrasilПолноценный мясной бразильс...,Развлечение
3,Бычьи хвосты в La bella società\n«Я решила про...,Развлечение
4,«Шанти»Ресторан — пионер паназиатской кухни в ...,Развлечение
...,...,...
26666,Актриса Агата Муцениеце немногословно ответила...,Развлечение
26667,Селин Дион в личном блоге опубликовала кадр со...,Развлечение
26668,Председатель Союза театральных деятелей России...,Развлечение
26669,Настасья Самбурская появится в неожиданном обр...,Развлечение


1. Разметка кластеризацией на основе самих текстов
2. Потом классификация на двух уровнях абстракции (первый работает быстро по саммаризации и бьет на более крупные классы), а второй точнее (по самому тексту на более точные классы). МЫ МОЖЕМ СДЕЛАТЬ НАЧАЛЬНЫХ КЛАССОВ ПОБОЛЬШЕ В РАЗНЫХ КОМБИНАЦИЯХ
3. Саммаризация текстов для извлечения признаков (возможно несколько саммаризаций как в датасете, то есть выделить описание, название статьи и ключевые слова)
4. Промты - дополняем текст так, чтобы он хорошо воспринмиался сетью. Один из полходов. М
5. Выделить сущности как фичи? 

Выделить сущности как признаки?

В интерфейсе выводить несколько дополнительных тегов (ближайших)
Как я и хотел, в итоге надо самостоятельно найти датасет и выделить классысамим, а дали нам только пример.

NER для фичей? Или можно в интерфейсе подсвечивать ключевые слова?



#### Запустим AutoML для классификации на данном нам датасете

Основа классификации текстов - это всегда эмбеддинги, так что нужно упор сделать в методы подбора эмбеддингов предложений (на основе эмбеддингов слов)
1. Weighted Average Transformer (WAT)
2. Bag of Random Embedding Projections (BOREP)
3. Random LSTM – LSTM со случайными весами
4. Bert Pooling – получение эмбеддинга с последнего выхода модели Transformer



In [4]:
from sklearn.model_selection import train_test_split

In [5]:
# Standard python libraries
import os
import time
import requests


# Installed libraries
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import torch

# Imports from our package
from lightautoml.automl.base import AutoML
from lightautoml.ml_algo.boost_lgbm import BoostLGBM
from lightautoml.ml_algo.boost_cb import BoostCB
from lightautoml.ml_algo.tuning.optuna import OptunaTuner
from lightautoml.automl.presets.text_presets import TabularNLPAutoML
from lightautoml.pipelines.features.lgb_pipeline import LGBSimpleFeatures
from lightautoml.pipelines.ml.base import MLPipeline
from lightautoml.pipelines.selection.importance_based import ImportanceCutoffSelector, ModelBasedImportanceEstimator
from lightautoml.reader.base import PandasToPandasReader
from lightautoml.tasks import Task
from lightautoml.automl.blend import WeightedBlender

In [6]:
RANDOM_STATE = 72
N_FOLDS = 5
N_THREADS = 4

roles = {
    'target' : 'target',
    'text' : 'text',
}

data_train, data_valid = train_test_split(my_df,  random_state=RANDOM_STATE, test_size=0.7)

In [7]:
task = Task('multiclass', metric='auc')
# ‘auc’ - ROC-AUC of each class against the rest.
# ‘auc_mu’ - AUC-Mu. Multi-class extension of standard AUC


automl = TabularNLPAutoML( 
    task=task,
    timeout=3600,
    cpu_limit=4,
    gpu_ids='0',
    general_params={  # За оптимизацию гиперпараметров отвечает 'nested_cv’: False
        'nested_cv': False,
        'use_algos': [['linear_l2', 'cb', 'nn']]  # nn - это файтюнинг берта
    },
    autonlp_params = {'sent_scaler' : 'l2',
                      'model_name': 'pooled_bert',
                      'transformer_params': {'model_params':
                                                 {'pooling': 'mean'},
                                             },
    },
    text_params={   # Change model?
        'lang': 'ru',
        'bert_model': 'prajjwal1/bert-tiny' #'google-bert/bert-base-uncased' #'prajjwal1/bert-tiny'  # Путь в библиотеке hugging face (БУКВАЛЬНО НАЗВАНИЕ В СПИСКЕ)
    },
    nn_params={
        'opt_params': {'lr': 1e-5},
        'max_length': 128,
        'bs': 32,
        'n_epochs': 10,
    },
    reader_params={
        'cv': N_FOLDS, 
        'random_state': RANDOM_STATE
    }
)

In [ ]:
oob_preds = automl.fit_predict(data_train, roles=roles, verbose=5)
print('OOB score: {}'.format(roc_auc_score(data_train[roles['target']].values, oob_preds.data, multi_class='ovr')))

[16:29:51] Stdout logging level is DEBUG.
[16:29:53] Model language mode: ru
[16:29:53] Task: multiclass

[16:29:53] Start automl preset with listed constraints:
[16:29:53] - time: 3600.00 seconds
[16:29:53] - CPU: 4 cores
[16:29:53] - memory: 16 GB

[16:29:53] Train data shape: (8001, 2)

[16:29:53] Layer 1 train process start. Time left 3599.99 secs
[16:30:02] Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...
[16:30:02] Training params: {'tol': 1e-06, 'max_iter': 100, 'cs': [1e-05, 5e-05, 0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10, 50, 100, 500, 1000, 5000, 10000, 50000, 100000], 'early_stopping': 2, 'categorical_idx': [], 'embed_sizes': (), 'data_size': 100}
[16:30:02] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_LinearL2 =====
[16:30:02] Linear model: C = 1e-05 score = 0.8936112954893621
[16:30:02] Linear model: C = 5e-05 score = 0.8936906424471951
[16:30:02] Linear model: C = 0.0001 score = 0.8937407809797402
[16:30:02] Linear model: C = 0.0005 score = 0.89

100%|██████████| 8/8 [00:32<00:00,  4.03s/it]

[16:30:41] Feature concated__text transformed
[16:30:42] Start fitting Lvl_0_Pipe_1_Mod_0_CatBoost ...
[16:30:42] Training params: {'task_type': 'GPU', 'thread_count': 4, 'random_seed': 42, 'num_trees': 3000, 'learning_rate': 0.03, 'l2_leaf_reg': 0.01, 'bootstrap_type': 'Bernoulli', 'grow_policy': 'SymmetricTree', 'max_depth': 5, 'min_data_in_leaf': 1, 'one_hot_max_size': 10, 'fold_permutation_block': 1, 'boosting_type': 'Plain', 'boost_from_average': True, 'od_type': 'Iter', 'od_wait': 100, 'max_bin': 32, 'feature_border_type': 'GreedyLogSum', 'nan_mode': 'Min', 'verbose': 100, 'allow_writing_files': False, 'devices': '0'}
[16:30:42] ===== Start working with fold 0 for Lvl_0_Pipe_1_Mod_0_CatBoost =====



Default metric period is 5 because AUC is/are not implemented for GPU
AUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


[16:30:42] 0:	test: 0.6491112	best: 0.6491112 (0)	total: 158ms	remaining: 7m 53s
[16:30:44] 100:	test: 0.8153717	best: 0.8161359 (95)	total: 2.12s	remaining: 1m
[16:30:46] 200:	test: 0.8325327	best: 0.8331750 (176)	total: 4.05s	remaining: 56.5s
[16:30:48] 300:	test: 0.8405305	best: 0.8405305 (300)	total: 6.01s	remaining: 53.9s
[16:30:50] 400:	test: 0.8481857	best: 0.8487311 (388)	total: 7.96s	remaining: 51.6s
[16:30:52] 500:	test: 0.8525743	best: 0.8525743 (500)	total: 9.95s	remaining: 49.6s
[16:30:54] 600:	test: 0.8580345	best: 0.8590394 (580)	total: 11.9s	remaining: 47.6s
[16:30:56] 700:	test: 0.8587568	best: 0.8601895 (649)	total: 13.9s	remaining: 45.5s
[16:30:57] bestTest = 0.8601894884
[16:30:57] bestIteration = 649
[16:30:57] Shrink model to first 650 iterations.
[16:30:57] ===== Start working with fold 1 for Lvl_0_Pipe_1_Mod_0_CatBoost =====
[16:30:58] 0:	test: 0.7010173	best: 0.7010173 (0)	total: 22.3ms	remaining: 1m 6s


...

[16:31:15] 100:	test: 0.8306924	best: 0.8306924 (100)	total: 2.02s	remaining: 58s
[16:31:17] 200:	test: 0.8576392	best: 0.8580320 (196)	total: 3.97s	remaining: 55.3s
[16:31:19] 300:	test: 0.8666815	best: 0.8685315 (281)	total: 5.91s	remaining: 53s
[16:31:21] 400:	test: 0.8728923	best: 0.8729843 (398)	total: 7.85s	remaining: 50.9s
[16:31:22] 500:	test: 0.8722423	best: 0.8755441 (422)	total: 9.77s	remaining: 48.8s
[16:31:23] bestTest = 0.8755440837
[16:31:23] bestIteration = 422
[16:31:23] Shrink model to first 423 iterations.
[16:31:23] ===== Start working with fold 3 for Lvl_0_Pipe_1_Mod_0_CatBoost =====
[16:31:24] 0:	test: 0.6760589	best: 0.6760589 (0)	total: 22.5ms	remaining: 1m 7s


Default metric period is 5 because AUC is/are not implemented for GPU
AUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


[16:31:26] 100:	test: 0.8085988	best: 0.8085988 (100)	total: 2.06s	remaining: 59.3s
[16:31:28] 200:	test: 0.8539931	best: 0.8543459 (193)	total: 4.02s	remaining: 56s
[16:31:29] 300:	test: 0.8651269	best: 0.8659902 (291)	total: 5.95s	remaining: 53.4s
[16:31:31] 400:	test: 0.8701555	best: 0.8703039 (398)	total: 7.92s	remaining: 51.4s
[16:31:33] 500:	test: 0.8702224	best: 0.8732550 (480)	total: 9.9s	remaining: 49.4s
[16:31:35] 600:	test: 0.8751740	best: 0.8751740 (600)	total: 11.9s	remaining: 47.3s
[16:31:37] 700:	test: 0.8773196	best: 0.8774346 (660)	total: 13.8s	remaining: 45.3s
[16:31:39] 800:	test: 0.8818692	best: 0.8818692 (800)	total: 15.9s	remaining: 43.5s
[16:31:41] 900:	test: 0.8824663	best: 0.8835932 (874)	total: 17.8s	remaining: 41.6s
[16:31:43] bestTest = 0.8835932487
[16:31:43] bestIteration = 874
[16:31:43] Shrink model to first 875 iterations.
[16:31:43] ===== Start working with fold 4 for Lvl_0_Pipe_1_Mod_0_CatBoost =====
[16:31:44] 0:	test: 0.6446133	best: 0.6446133 (0)	t

Default metric period is 5 because AUC is/are not implemented for GPU
AUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


[16:31:46] 100:	test: 0.8116062	best: 0.8116062 (100)	total: 2.57s	remaining: 1m 13s
[16:31:48] 200:	test: 0.8332051	best: 0.8350637 (168)	total: 4.66s	remaining: 1m 4s
[16:31:50] 300:	test: 0.8394760	best: 0.8400030 (299)	total: 6.68s	remaining: 59.9s
[16:31:53] 400:	test: 0.8442819	best: 0.8447486 (379)	total: 8.79s	remaining: 57s
[16:31:54] 500:	test: 0.8452699	best: 0.8467971 (470)	total: 10.8s	remaining: 53.6s
[16:31:56] bestTest = 0.8467970552
[16:31:56] bestIteration = 470
[16:31:56] Shrink model to first 471 iterations.
[16:31:56] Fitting Lvl_0_Pipe_1_Mod_0_CatBoost finished. score = 0.8737941205303646
[16:31:56] Lvl_0_Pipe_1_Mod_0_CatBoost fitting and predicting completed
[16:31:56] Time left 3476.83 secs
[16:31:56] number of text features: 1 
[16:31:56] number of categorical features: 0 
[16:31:56] number of continuous features: 0 
[16:31:56] Start fitting Lvl_0_Pipe_2_Mod_0_TorchNN__linear_layer_0 ...
[16:31:56] Training params: {'num_workers': 4, 'pin_memory': False, 'max_l

In [70]:
oob_preds.data[:, 0]

array([0.8013454 , 0.23664728, 0.29297793, ..., 0.03079473, 0.03808406,
       0.12075672], dtype=float32)

In [ ]:
print('OOB score: {}'.format(roc_auc_score(data_train[roles['target']].values, oob_preds.data, multi_class="ovr")))
test_preds = automl.predict(data_valid, n_jobs=N_THREADS)
print(f'TEST score: {roc_auc_score(data_valid[roles["target"]].values, test_preds.data, multi_class="ovr")}')

In [87]:
print(f'TEST score: {roc_auc_score(data_valid[roles["target"]].values, test_preds.data, multi_class="ovr")}')

TEST score: 0.6144871049459633


#### Выберем хорошие подходы для классификации текстов.

##### BERT Hugging Face

In [96]:
from transformers import AutoModelForSequenceClassification
import evaluate
from transformers import TrainingArguments, Trainer


In [97]:
model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-uncased", num_labels=4)

#  DeepPavlov/rubert-base-cased, google-bert/bert-base-uncased

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [99]:
metric = evaluate.load("accuracy")
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=data_train,
    eval_dataset=data_valid,
    compute_metrics=compute_metrics
)

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

In [ ]:
trainer.train()